
- The class is designed to be memory-efficient by using sparse matrices, which is particularly useful for datasets with many groups or sparse random effects design matrices.
- The log-likelihood calculation is performed group by group, which allows for efficient computation even with large numbers of groups.
- The use of L-BFGS-B optimization method allows for bounded optimization, which can help in ensuring that variance components remain positive.
- The current implementation assumes that random effects are independent across groups. More complex covariance structures for random effects are not implemented in this version.
- The prediction method currently only uses fixed effects. Incorporating random effects into predictions would require additional computation and would depend on the specific use case.
- The class doesn't currently provide standard errors for the estimates. Implementing this would require computing the Hessian matrix or using a method like bootstrapping.
- Diagnostic tools, such as residual plots or influence measures, are not included in this implementation but could be valuable additions for assessing model fit and assumptions.


In [168]:
import numpy as np
from scipy import sparse
from scipy.optimize import minimize
import scipy.sparse.linalg as spla
from sklearn.preprocessing import StandardScaler

class MixedLM:
    def __init__(self, endog, exog, groups, exog_re=None):
        self.endog = np.asarray(endog).ravel()
        self.exog = sparse.csr_matrix(exog)
        self.groups = np.asarray(groups)
        self.exog_re = sparse.csr_matrix(exog_re) if exog_re is not None else sparse.eye(self.exog.shape[0], format='csr')
        
        self.unique_groups = np.unique(groups)
        self.n_groups = len(self.unique_groups)
        self.n_re = self.exog_re.shape[1]
        self.n_fe = self.exog.shape[1]
        
        # Standardize the fixed effects features
        self.scaler = StandardScaler(with_mean=False)
        self.exog = sparse.csr_matrix(self.scaler.fit_transform(self.exog.toarray()))
        
        # Pre-compute group-wise matrices
        self.group_matrices = self._precompute_group_matrices()

    def _precompute_group_matrices(self):
        group_matrices = {}
        for group in self.unique_groups:
            mask = self.groups == group
            group_matrices[group] = {
                'X': self.exog[mask],
                'Z': self.exog_re[mask],
                'y': self.endog[mask]
            }
        return group_matrices

    def log_likelihood_and_gradient(self, params):
        beta = params[:self.n_fe]
        sigma_re = np.exp(params[self.n_fe:self.n_fe + self.n_re])
        sigma_e = np.exp(params[-1])

        ll = 0
        grad = np.zeros_like(params)

        for group, matrices in self.group_matrices.items():
            X, Z, y = matrices['X'], matrices['Z'], matrices['y']
            
            V = Z @ sparse.diags(sigma_re**2) @ Z.T + sigma_e**2 * sparse.eye(Z.shape[0])
            r = y - X @ beta

            # Use sparse Cholesky decomposition
            try:
                chol = spla.cholesky(V.tocsc())
            except:
                # If Cholesky fails, use LU decomposition as a fallback
                lu = spla.splu(V.tocsc())
                V_inv_r = lu.solve(r)
                log_det = np.sum(np.log(np.abs(lu.U.diagonal())))
            else:
                V_inv_r = chol.solve_A(r)
                log_det = 2 * np.sum(np.log(chol.L().diagonal()))
            
            # Log-likelihood
            ll += -0.5 * (r.T @ V_inv_r + log_det)

            # Gradient components
            V_inv_X = chol.solve_A(X.toarray()) if 'chol' in locals() else lu.solve(X.toarray())
            grad[:self.n_fe] += X.T @ V_inv_r
            
            V_inv = chol.solve_A(sparse.eye(V.shape[0]).toarray()) if 'chol' in locals() else lu.solve(sparse.eye(V.shape[0]).toarray())
            P = V_inv - V_inv_r[:, np.newaxis] @ V_inv_r[np.newaxis, :]
            
            for j in range(self.n_re):
                W_j = Z[:, j].toarray() @ Z[:, j].toarray().T
                grad[self.n_fe + j] += 0.5 * np.trace(P @ W_j) * sigma_re[j]**2
            
            grad[-1] += 0.5 * np.trace(P) * sigma_e**2

        # Add L2 regularization for random effects
        lambda_reg = 0.1
        ll -= 0.5 * lambda_reg * np.sum(sigma_re**2)
        grad[self.n_fe:-1] -= lambda_reg * sigma_re**2

        return -ll, -grad

    def fit(self):
        # Initialize fixed effects using ridge regression with sparse matrices
        lambda_ridge = 0.1
        A = self.exog.T @ self.exog + lambda_ridge * sparse.eye(self.n_fe)
        b = self.exog.T @ self.endog
        ridge_beta = spla.spsolve(A, b)
        
        # Initialize variance components
        initial_re_var = np.var(self.endog) / (self.n_re + 1)
        initial_params = np.concatenate([
            ridge_beta,
            np.log(np.random.uniform(0.5, 1.5, self.n_re) * initial_re_var),
            [np.log(initial_re_var)]
        ])

        # Set bounds
        bounds = [(None, None)] * self.n_fe + [(-10, 10)] * (self.n_re + 1)

        result = minimize(
            fun=lambda x: self.log_likelihood_and_gradient(x)[0],
            x0=initial_params,
            method='L-BFGS-B',
            jac=lambda x: self.log_likelihood_and_gradient(x)[1],
            bounds=bounds,
            options={'maxiter': 100, 'ftol': 1e-6, 'gtol': 1e-6}
        )
        self.params = result.x
        return result

    def predict(self, exog_new):
        beta = self.params[:self.n_fe]
        exog_new_scaled = self.scaler.transform(exog_new)
        return exog_new_scaled @ beta

In [169]:


# Dimensionality Reduction:
# With one-hot encoding for all players at each position, you'd end up with a very high-dimensional dataset. Mixed-effects models can handle this more efficiently by treating players as random effects.
# Partial Pooling:
# Mixed-effects models perform partial pooling, which is particularly useful for players with limited data. It balances between individual estimates and overall position averages.
# Hierarchical Structure:
# You can model the nested structure of the data: plays within games, players within positions, etc.
# Handling of Repeated Measures:
# Since the same players appear in multiple plays, mixed-effects models can account for this non-independence in the data.
# Separation of Fixed and Random Effects:
# You can model position-specific effects as fixed, while individual player deviations are treated as random.



In [193]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.cross_decomposition import PLSRegression

# Assume we have a DataFrame 'df' with columns: 'team', 'opponent', 'is_home', 'team_score'

# Step 1: Prepare the data
def prepare_data(df):
    # Split the data
    X = df[['team', 'opponent', 'is_home']]
    y = df['team_score']
    return train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = prepare_data(season_2022)

# Step 2: Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), ['team', 'opponent']),
        ('num', StandardScaler(), ['is_home'])
    ])

# Step 3: Create and train the FM-like model
# We'll use PLSRegression as an approximation of FM
fm_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', PLSRegression(n_components=10, max_iter=1000))
])

fm_model.fit(X_train, y_train)

# Step 4: Make predictions
y_pred = fm_model.predict(X_test)

# Step 5: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

# Step 6: Extract team effects and home advantage
# Get feature names after one-hot encoding
feature_names = (fm_model.named_steps['preprocessor']
                 .named_transformers_['cat']
                 .get_feature_names_out(['team', 'opponent']))
feature_names = np.concatenate([feature_names, ['is_home']])

# Extract coefficients
coef = fm_model.named_steps['regressor'].coef_

# Team effects
team_effects = coef[0, :len(feature_names)//2 - 1]  # Exclude 'opponent' features
opponent_effects = coef[0, len(feature_names)//2 - 1:-1]  # Exclude 'is_home' feature
home_advantage = coef[0, -1]

# Normalize effects
team_effects -= np.mean(team_effects)
opponent_effects -= np.mean(opponent_effects)

# Get team names
team_names = (fm_model.named_steps['preprocessor']
              .named_transformers_['cat']
              .categories_[0][1:])  # Exclude the dropped category

# Print results
print("Home Advantage:", home_advantage)
print("Team Effects:", dict(zip(team_names, team_effects)))
print("Opponent Effects:", dict(zip(team_names, opponent_effects)))

# Optionally, you can analyze interactions between teams
def team_interaction(team1, team2):
    team1_idx = np.where(team_names == team1)[0][0]
    team2_idx = np.where(team_names == team2)[0][0]
    return np.dot(fm_model.named_steps['regressor'].x_scores_[:, team1_idx], 
                  fm_model.named_steps['regressor'].y_scores_[:, team2_idx])

# Example: Interaction between first two teams
print("Interaction between", team_names[0], "and", team_names[1], ":", 
      team_interaction(team_names[0], team_names[1]))

Mean Squared Error: 103.55039650898577
R-squared Score: 0.2770650158011573
Home Advantage: 1.1102719554835345
Team Effects: {'Air Force': np.float64(-9.097774951553317), 'Akron': np.float64(-2.5167905319321364), 'Alabama': np.float64(14.956583122700662), 'Alabama A&M': np.float64(-14.329043830838451), 'Alabama St': np.float64(-1.055467605671321), 'Alcorn St': np.float64(-3.0140821741232786), 'American Univ': np.float64(-10.105719083737773), 'Appalachian St': np.float64(-3.6599816408379033), 'Arizona': np.float64(18.243022824286378), 'Arizona St': np.float64(-0.5215705419635137), 'Ark Little Rock': np.float64(-5.049539994089337), 'Ark Pine Bluff': np.float64(-5.312086408371637), 'Arkansas': np.float64(10.503664509473392), 'Arkansas St': np.float64(-0.2868000201234972), 'Army': np.float64(-2.6966644898708756), 'Auburn': np.float64(10.382009196255238), 'Austin Peay': np.float64(-8.292153137307755), 'BYU': np.float64(6.3368310938326875), 'Ball St': np.float64(-0.032369047812175145), 'Baylo

In [185]:
!pip install xlearn

     ---------------------------------------- 0.0/4.9 MB ? eta -:--:--
     ------ --------------------------------- 0.8/4.9 MB 8.5 MB/s eta 0:00:01
     ------------------------------- -------- 3.9/4.9 MB 13.1 MB/s eta 0:00:01
     ---------------------------------------- 4.9/4.9 MB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for xlearn: filename=xlearn-0.40a1-py3-none-any.whl size=11116 sha256=e747bbca203ae7974136a7abc86f1c8fb0614eb9d2201fb24127762d6bbcb5cb
  Stored in directory: c:\users\blake\appdata\local\pip\cache\wheels\72\26\6e\2972274d133f591c34d9299ee113ee5c66bd1b2b1a79006789
Successfully built xlearn


In [170]:


DATA_PATH = '../data/testing/ncaam_sample_data.csv'
def load_data(data_path):
    return pd.read_csv(data_path)

m_data = load_data(DATA_PATH)
scaler = MinMaxScaler()
m_data['team_score']= m_data['team_score'].clip(36, 120)
m_data['continuous_target'] = scaler.fit_transform(m_data['team_score'].values.reshape(-1, 1))
m_data['continuous_target_2'] = m_data['team_fgm'].copy()/m_data['team_fga'].copy()
m_data['date'] = pd.to_datetime(m_data['date'])
m_data = m_data.sort_values(by=['date', 'team_name']).reset_index(drop=True)
m_data = m_data.rename(columns={'team_name':'team','opp_name':'opponent'})

In [171]:
m_data.head()

,season,team_score,opp_score,is_home,numot,team_fgm,team_fga,team_fgm3,team_fga3,team_ftm,...,opp_ast,opp_to,opp_stl,opp_blk,opp_pf,team,opponent,date,continuous_target,continuous_target_2
0,2003,68,62,0,0,27,58,3,14,11,...,8,18,9,2,20,Alabama,Oklahoma,2002-11-14,0.380952,0.465517
1,2003,70,63,0,0,26,62,8,20,10,...,7,12,8,6,16,Memphis,Syracuse,2002-11-14,0.404762,0.419355
2,2003,62,68,0,0,22,53,2,10,16,...,13,23,7,1,22,Oklahoma,Alabama,2002-11-14,0.309524,0.415094
3,2003,63,70,0,0,24,67,6,24,9,...,16,13,4,4,18,Syracuse,Memphis,2002-11-14,0.321429,0.358209
4,2003,55,81,-1,0,20,46,3,11,12,...,12,9,9,3,18,E Washington,Wisconsin,2002-11-15,0.226190,0.434783


In [172]:
season_2022 = m_data[m_data['season'] == 2022].copy().reset_index(drop=True)

In [173]:
import numpy as np
import pandas as pd
from scipy import stats

def generate_basketball_data(n_games=1000, n_teams=30):
    np.random.seed(42)  # for reproducibility
    
    # Generate team names
    teams = [f"Team_{i}" for i in range(n_teams)]
    
    # Generate team strengths (will be used for both offense and defense)
    team_strengths = stats.norm.rvs(loc=0, scale=5, size=n_teams)
    
    # Generate data
    data = []
    for _ in range(n_games):
        home_team = np.random.choice(teams)
        away_team = np.random.choice([t for t in teams if t != home_team])
        
        home_strength = team_strengths[teams.index(home_team)]
        away_strength = team_strengths[teams.index(away_team)]
        
        # Home advantage (randomly between 2 to 4 points)
        home_advantage = np.random.uniform(2, 4)
        
        # Generate scores (base score + team strength + random noise)
        base_score = 80
        home_score = int(base_score + home_strength + home_advantage + np.random.normal(0, 10))
        away_score = int(base_score + away_strength + np.random.normal(0, 10))
        
        # Ensure non-negative scores
        home_score = max(home_score, 0)
        away_score = max(away_score, 0)
        
        # Additional features
        days_rest_home = np.random.randint(1, 5)
        days_rest_away = np.random.randint(1, 5)
        
        data.append({
            'home_team': home_team,
            'away_team': away_team,
            'home_score': home_score,
            'away_score': away_score,
            'days_rest_home': days_rest_home,
            'days_rest_away': days_rest_away
        })
    
    return pd.DataFrame(data)

# Generate the data
basketball_df = generate_basketball_data()

# Display the first few rows
print(basketball_df.head())

# Basic statistics
print(basketball_df.describe())

# Save to CSV (optional)
basketball_df.to_csv('basketball_scores.csv', index=False)

  home_team away_team  home_score  away_score  days_rest_home  days_rest_away
0   Team_19   Team_28          75          66               3               3
1   Team_13   Team_17          60          92               2               2
2    Team_9    Team_3          92          89               4               2
3   Team_22   Team_28          72          61               2               3
4   Team_20   Team_24          93          59               1               1
        home_score   away_score  days_rest_home  days_rest_away
count  1000.000000  1000.000000     1000.000000     1000.000000
mean     81.572000    78.766000        2.498000        2.548000
std      11.048932    11.054476        1.117696        1.120246
min      51.000000    39.000000        1.000000        1.000000
25%      74.000000    71.000000        1.750000        2.000000
50%      81.000000    79.000000        2.000000        3.000000
75%      89.000000    86.000000        3.000000        4.000000
max     118.000000  

In [174]:
def prepare_data_for_mixedlm(df):
    y = df['home_score'] - df['away_score']
    X = np.column_stack([
        df['days_rest_home'],
        df['days_rest_away'],
        np.ones(len(df))  # Intercept
    ])
    groups = df['home_team'].astype('category').cat.codes
    
    # We don't need to create Z here, as the class will handle it internally
    return y, X, groups

# Assuming basketball_df is already created
y, X, groups = prepare_data_for_mixedlm(basketball_df)

# Create and fit the model
model = MixedLM(y, X, groups)

start_time = time.time()
result = model.fit()
end_time = time.time()

In [175]:
result

  message: ABNORMAL_TERMINATION_IN_LNSRCH
  success: False
   status: 2
      fun: 1101759.854522504
        x: [-2.562e-01  2.630e-01 ... -1.451e+00 -1.446e+00]
      nit: 0
      jac: [ 1.528e+03 -1.115e+03 ...  1.170e+02  6.150e+05]
     nfev: 21
     njev: 21
 hess_inv: <1004x1004 LbfgsInvHessProduct with dtype=float64>